# Plota série temporal de tempertura máxima de estações meteorológicas do INMET

- Reportagem sobre altas temperaturas em dezembro de 2025 no Estado de SP: https://g1.globo.com/sp/sao-paulo/noticia/2025/12/28/sp-registra-novo-recorde-de-calor-para-o-mes-de-dezembro-com-372oc.ghtml

- Estação utilizada: estação do Mirante de Santana - A701. Nos dados históricos tem disponível o ano completo desde 2007. Embora no site tem arquivos de estações desde 2000. São processados 19 anos de dados (2007 à 2025).
    - REGIÃO: SE
    - UF: SP
    - ESTAÇÃO: SAO PAULO - MIRANTE
    - CODIGO (WMO): A701
    - LATITUDE: -23,48333333
    - LONGITUDE: -46,61666666
    - ALTITUDE: 792,06
    - DATA DE FUNDAÇÃO (YYYY-MM-DD): 2006-07-25

- Dados últimos 6 meses: https://bdmep.inmet.gov.br/

- Dados série histórica: https://portal.inmet.gov.br/dadoshistoricos

- Realizado por: Enrique V. Mattos - 30/12/2025

# **1° Passo:** Preparando ambiente

In [ ]:
# instalações
!pip install -q ultraplot

# monta o drive
from google.colab import drive
drive.mount('/content/drive')

# nome do diretório
dir = '/content/drive/MyDrive/PYHTON/00_GITHUB/16_LINKEDIN/09_estacoes_meteorologicas'

# importa bibliotecas
import ultraplot as uplt
import pandas as pd
import zipfile
import glob
import numpy as np
import matplotlib.ticker as ticker
import warnings
warnings.filterwarnings("ignore")

# **Processando**

In [ ]:
#======================================================================================#
#                              DEFINE A ESTAÇÃO
#======================================================================================#
codigo_estacao = 'A701'
nome_estacao = 'Mirante de Santana'

#======================================================================================#
#                        DADOS ANTERIORES A DEZEMBRO/2025
#======================================================================================#
# Fonte dos dados históricos: https://portal.inmet.gov.br/dadoshistoricos

# Loop nos anos
files = []
dfs = []
for ano in range(2007, 2025+1):

    print('DESCOMPACTANDO ANO === >>>', ano)

    # abre o arquivo ZIP e extrai tudo
    with zipfile.ZipFile(f'/{dir}/input/{str(ano)}.zip', 'r') as zip_ref:
        zip_ref.extractall("/content/")

    # appenda os arquivos
    if ano <= 2019:
        file = sorted(glob.glob(f'/content/{str(ano)}/INMET*{codigo_estacao}*'))[0]
    else:
        file = sorted(glob.glob(f'/content/INMET*{codigo_estacao}*{ano}.CSV'))[0]

    # leitura do arquivo
    df = pd.read_csv(file,
                     encoding='iso-8859-1',
                     skiprows=8,
                     decimal=',',
                     delimiter=';')

    # Os arquivos de "2007 à 2018" a coluna data e hora estão no seguinte formato:
    # DATA (YYYY-MM-DD) HORA (UTC)
    #    2015-01-01	      00:00

    # Os arquivos depois de "2019" a coluna data e hora estão no seguinte formato:
    #    Data        Hora UTC
    #  2019/01/01	 0000 UTC

    # se ano entre 2007 e 2018:
    if 2007 <= ano <= 2018:
        df.rename(columns={'DATA (YYYY-MM-DD)': 'data',
                           'HORA (UTC)': 'hora',
                           'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'tmax'}, inplace=True)

        df = df[['data', 'hora', 'tmax']]
        df['data'] = df['data'].astype(str).str.replace('-', '/', regex=False)
        df['data_hora'] = df['data'] + ' ' + df['hora'].astype(str).str.replace(':', '', regex=False).str.zfill(4)
        df.drop(columns=['data','hora'], inplace=True)
        df.dropna(inplace=True)

    else:
        df.rename(columns={'Data': 'data',
                           'Hora UTC': 'hora',
                           'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'tmax'}, inplace=True)

        df = df[['data', 'hora', 'tmax']]
        df['data_hora'] = df['data'] + ' ' + df['hora'].astype(str).str.replace(' UTC', '', regex=False).str.zfill(4)
        df.drop(columns=['data','hora'], inplace=True)
        df.dropna(inplace=True)

    # adiciona o DataFrame à lista inicial
    dfs.append(df)

# concatenar DataFrames
df_antes_nov2025 = pd.concat(dfs, ignore_index=True)

# transforma a coluna "data_hora" em índice da tabela
df_antes_nov2025['data_hora'] = pd.to_datetime(df_antes_nov2025['data_hora'], format='%Y/%m/%d %H%M')

# definir como índice
df_antes_nov2025.set_index('data_hora', inplace=True)

#======================================================================================#
#                             DADOS DE DEZEMBRO/2025
#======================================================================================#
# Fonte dos dados: https://bdmep.inmet.gov.br/

# leitura dos dados
df_dez2025 = pd.read_csv(f'{dir}/input/generatedBy_react-csv.csv', sep=';', usecols=['Data', 'Hora (UTC)', 'Temp. Max. (C)'])

# concatenar "Data" e "Hora UTC" e converter para datetime
df_dez2025['data_hora'] = df_dez2025['Data'] + ' ' + df_dez2025['Hora (UTC)'].astype(str).str.zfill(4)
df_dez2025['data_hora'] = pd.to_datetime(df_dez2025['data_hora'], format='%d/%m/%Y %H%M')

# definir como índice
df_dez2025.set_index('data_hora', inplace=True)

# remover colunas antigas
df_dez2025.drop(columns=['Data','Hora (UTC)'], inplace=True)

# remover NaN
df_dez2025.dropna(inplace=True)

# renomear coluna de Temperatura
df_dez2025.rename(columns={'Temp. Max. (C)': 'tmax'}, inplace=True)

# converção da coluna 'tmax' para numérica, substituindo ',' por '.'
df_dez2025['tmax'] = df_dez2025['tmax'].str.replace(',', '.', regex=False).astype(float)

#======================================================================================#
#                             JUNTAR OS DADOS
#======================================================================================#
# juntar dataframes
df_merge = pd.concat([df_antes_nov2025, df_dez2025])

# remover linhas com -9999.0
df_merge = df_merge[df_merge['tmax'] != -9999.0]

#======================================================================================#
#                             AGRUPA POR DIA e POR MÊS
#======================================================================================#
# agrupa por dia, usando resample
df_dia = df_merge.resample('D').max()

# agrupa por mês, usando resample
df_mes = df_merge.resample('M').max()

In [ ]:
# dados de dezembro de 2025
df_dez2025

In [ ]:
# dado de 1 ano
df

In [ ]:
# todos os dados
df_merge

In [ ]:
# dados por dia
df_dia

In [ ]:
# dados por mês
df_mes

# **Plota figura**: DEZ 2025

In [ ]:
df_dia_2025 = df_dia['2025':'2025']
df_dia_2025

In [ ]:
# cria a moldura da figura
fig, ax = uplt.subplots(figsize=(10,5), tight=True)

# plota gráfico de barras
ax.bar(df_dia_2025['tmax'].index,
       df_dia_2025['tmax'],
       color='blue')

# encontra o índice do dataframe respectivo do dia 23/08/2025
dia_x = '2025-12-28'
indice = df_dia.index.get_loc(dia_x)

# chuva do dia 2025-12-28
tmax_dia28 = round(df_dia.loc[dia_x]['tmax'], 1)

# linha vertical indicando o dia 2025-12-28
ax.axvline(x=df_dia.index[indice], color='bright red', linestyle='--', lw=1.2)

# anotação da linha vertical
ax.annotate(f"{dia_x} \n   ({tmax_dia28}°C)", xy=(df_dia.index[indice-30], 37.8), fontsize=10, color='bright red',
             bbox=dict(boxstyle="round", fc="0.8"))

# datas
datai = str(df_dia_2025.index[0].year) + '-' + str(df_dia_2025.index[0].month).zfill(2) + '-' +  str(df_dia_2025.index[0].day).zfill(2)
dataf = str(df_dia_2025.index[-1].year) + '-' + str(df_dia_2025.index[-1].month).zfill(2) + '-' +  str(df_dia_2025.index[-1].day).zfill(2)

# formato dos eixos
ax.format(title=f'Temperatura Máxima (°C)\n',
          titleloc='l',
          titleweight='bold',
          titlecolor='red',
          xlabel='Ano-Mês-Dia',
          ylabel='Temperatura',
          small='20px',
          large='23px',
          xrotation=30,
          ytickminor=False,
          ylim=(20,38),
          yformatter=ticker.FormatStrFormatter('%.1f'))

# plota subtítulo
ax.text(pd.to_datetime('2024-12-14'), 38.2, f'Período: {datai} à {dataf} / Estação: {nome_estacao} - {codigo_estacao}',
        color='gray', fontsize=11)

# salva figura
fig.save(f'{dir}/output/Fig_1_diario_tmax_2025_{codigo_estacao}.jpg', dpi=300)

# **Plota figura**: MENSAL DESDE 2020

In [ ]:
# dados mensais que iremos utilizar
df_mes

In [ ]:
#========================================================================================================================#
#                                          ACUMULA PARA MENSAL
#========================================================================================================================#
# calculando a média mensal
df_mensal_climatologia = df_mes.groupby(df_mes.index.month).mean()

#========================================================================================================================#
#                                                PLOTA FIGURA
#========================================================================================================================#
# moldura da figura
fig, ax = uplt.subplots(figsize=(10,5), tight=True)

# 2007-2019
anoi, anof = 2007, 2019
cycle = uplt.Cycle('Greys', left=0.2, N=13)
for ano in np.arange(anoi, anof+1):
    ax.plot(uplt.arange(1, 12, 1),
            df_mes.loc[str(ano), 'tmax'].values,
            label=str(ano),
            cycle=cycle,
            ls='--',
            lw=1)

# 2020-2024
anoi, anof = 2020, 2024
cycle = uplt.Cycle('jet', left=0.2, N=5)
for ano in np.arange(anoi, anof+1):
    ax.plot(uplt.arange(1, 12, 1),
            df_mes.loc[str(ano), 'tmax'].values,
            label=str(ano),
            cycle=cycle,
            ls='-',
            lw=2)

# 2025
val2025 = df_mes.loc['2025', 'tmax'].values
ax.plot(uplt.arange(1, 12, 1),
        val2025,
        label='2025',
        color='bright red',
        ls='-',
        lw=4,
        marker='o',                   # Forma do símbolo
        markersize=15,                # Tamanho
        markerfacecolor='white',      # Cor interna
        markeredgecolor='bright red', # Cor da borda
        markeredgewidth=2)            # Espessura da borda)

# mensal climatologia
ax.plot(uplt.arange(1, 12, 1),
        df_mensal_climatologia['tmax'].values,
        label='Média',
        color='black',
        ls='--',
        lw=3)

# linha vertical indicando dez/2025
ax.axvline(x=12, color='bright red', linestyle='--', lw=1.2)

# anotação da linha vertical
ax.annotate(f"Dez./2025 \n   ({tmax_dia28}°C)",
            xy=(11.5, 38.5),
            fontsize=10,
            color='bright red',
            bbox=dict(boxstyle="round", fc="0.8"))

# formato dos eixos
ax.format(title=f'Temperatura Máxima (°C)\n',
          titleloc='l',
          titleweight='bold',
          titlecolor='red',
          ylabel='Temperatura',
          small='20px',
          large='25px',
          xlabel='Mês', xticks=1,
          xticklabels=['', 'Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'],
          xrotation=0, xtickminor=False, ytickminor=False, grid=False,
          xlim=(0.5, 12.5),
          ylim=(25,40),
          yformatter=ticker.FormatStrFormatter('%.0f'))

# plota subtítulo
datai = str(df_mes.index[0].year)
dataf = str(df_mes.index[-1].year)
ax.text(0.5, 40.3, f'Período: {datai} à {dataf} / Estação: {nome_estacao} - {codigo_estacao}',
        color='gray', fontsize=11)

# legenda
ax.legend(loc='best', ncols=6, frameon=True, prop={'size':8})

# salva figura
fig.save(f'{dir}/output/Fig_2_mensal_tmax_2007_a_2025_{codigo_estacao}.jpg', dpi=300)